In [65]:
import sklearn as sk
import numpy as np
import pandas as pd
import time
from sklearn.cross_validation import KFold
from sklearn import tree
from sklearn import ensemble
from sklearn import datasets

import matplotlib.pylab as plt
%matplotlib inline

In [66]:
class CART:    
    def __init__(self, features_rate, min_samples_leaf, max_depth):
        self.features_rate = features_rate
        self.tree_map = {}
        self.min_samples_leaf = min_samples_leaf
        if (max_depth == None):
            self.max_depth = 100000
        else:
            self.max_depth = max_depth

    def gen_tree(self, element_numbers, vertex_num, mean, now_depth, mse_value):
         
        if (now_depth == self.max_depth):  
            self.tree_map[str(vertex_num)] = [[None, None], [1], [mean], [None, None]]
            return 0
        
        min_mse = mse_value

        pos_num_split = np.nan
        value_split = np.nan
        
        left = []
        right = []
        
        cou_elements = len(element_numbers)
        
        best_left_square_sum = np.nan
        best_left_sum = np.nan
        best_right_square_sum = np.nan
        best_right_sum = np.nan
        
        x = self.x[element_numbers]
        y = self.y[element_numbers]

        for feature_num in range(self.features_cou_analize):

            sort_ind = x[:,feature_num].argsort()
            new_x = x[:,feature_num][sort_ind]
            new_y = y[sort_ind]
            
            left_cou = 0
            right_cou = cou_elements
            
            del_num = 0
            right_square_sum = mse_value + mean * mean * cou_elements * 1.0
            right_sum = mean * cou_elements * 1.0
            left_square_sum = 0.0
            left_sum = 0.0
            feature_value_split = np.nan
            for i, feature_value in enumerate(new_x):
                
                if (feature_value_split == feature_value):
                    continue
                    
                if (np.isnan(feature_value_split)):
                    feature_value_split = feature_value
                else:
                    feature_value_split = 0.5 * (feature_value_split + feature_value)
                    
                while (1):
                    if (new_x[del_num] < feature_value_split):
                        y_val = new_y[del_num]
                        del_num += 1
                        left_cou += 1
                        right_cou -= 1
                        
                        left_square_sum += y_val * y_val
                        left_sum += y_val
                        
                        right_square_sum -= y_val * y_val
                        right_sum -= y_val
                    else:
                        break
                
                if (left_cou < self.min_samples_leaf):
                    continue
                elif (right_cou < self.min_samples_leaf):
                    break
                
                now_left_mse = left_square_sum - left_sum * left_sum * 1.0/ left_cou
                now_right_mse = right_square_sum - right_sum * right_sum * 1.0 / right_cou
                
                if (now_left_mse + now_right_mse < min_mse):
                    min_mse = now_left_mse + now_right_mse
                    pos_num_split = feature_num
                    value_split = feature_value_split
                    
                    best_left_square_sum = left_square_sum
                    best_left_sum = left_sum
                    
                    best_right_square_sum = right_square_sum
                    best_right_sum = right_sum
                    
        if (np.isnan(value_split)):
            self.tree_map[str(vertex_num)]= [[None, None], [1], [mean], [None, None]]
            return 0
        
        self.tree_map[str(vertex_num)] = [[self.feature_order[pos_num_split], value_split], [0], [mean], [vertex_num*2 + 1, vertex_num*2 + 2]]
        
        
        left_class = []
        right_class = []
        
        for pos in element_numbers:
            if (self.x[pos][pos_num_split] < value_split):
                left.append(pos)
            else:
                right.append(pos)

        left_mse = best_left_square_sum - (best_left_sum ** 2.0) / len(left)
        right_mse = best_right_square_sum - (best_right_sum ** 2.0) / len(right)
        
        self.feature_contribution[self.feature_order[pos_num_split]] += (mse_value - left_mse - right_mse)
        
        self.gen_tree(left, vertex_num*2 + 1, best_left_sum * 1.0 / len(left), now_depth + 1, left_mse)
        self.gen_tree(right, vertex_num*2 + 2, best_right_sum *1.0/ len(right), now_depth + 1, right_mse)
        
    def fit(self, x, y):
        self.tree_map = {}
        
        x = np.asarray(x, dtype = np.float)
        features_cou = (x.shape)[1]
        self.feature_order = np.sort(np.random.choice((x.shape)[1], int((x.shape)[1] * (self.features_rate)), replace=False))
        
        self.features_cou_analize = int((x.shape)[1] * (self.features_rate))
        self.cou_samples = len(x)
        
        self.feature_contribution = np.zeros(features_cou)
        
        for i in reversed(range(features_cou)):
            if (i not in self.feature_order):
                x = np.delete(x, i, 1)

        self.x = x
        self.y = np.asarray(y, dtype = np.float)
        y_pred = np.tile(np.sum(y) * 1.0 / len(y), len(y))
        
        self.gen_tree(np.arange(self.cou_samples), 0, np.sum(y) * 1.0 / len(y), 0, np.sum((y_pred - self.y) ** 2.0))
        
        return self.tree_map

    def predict(self, data):
        y_pred = []
        for sample in data:
            now_vertex = 0
            while (1):
                if (self.tree_map[str(now_vertex)][1][0] == 1):
                    break
                if (sample[self.tree_map[str(now_vertex)][0][0]] >= self.tree_map[str(now_vertex)][0][1]):
                    now_vertex = self.tree_map[str(now_vertex)][3][1]
                else:
                    now_vertex = self.tree_map[str(now_vertex)][3][0]
            y_pred.append(self.tree_map[str(now_vertex)][2][0])
        return y_pred
        
        

In [67]:
f = open("spam.train.txt", "r")
y_true = []
data = []
for line in f:
    line = line[:len(line) - 1]
    arr = line.split(' ')
    y_true.append(arr[0])
    del arr[0]
    data.append(arr)
x = np.asarray(data, dtype = np.float)
y = np.asarray(y_true, dtype = np.float)

In [68]:
f = open("spam.test.txt", "r")
test_data = []
test_y = []
for line in f:
    line = line[:len(line) - 1]
    arr = line.split(' ')
    test_y.append(arr[0])
    del arr[0]
    test_data.append(arr)
    
test_x = np.asarray(test_data, dtype = np.float)
test_y = np.asarray(test_y, dtype = np.float)

In [69]:
class modify_gb:
    
    def __init__(self, n_estimators, features_rate, cou_samples, min_samples_split, learning_rate, max_depth, loss, regularizator_value):
        self.regularizator_value = regularizator_value
        self.number_elements_in_subsample = cou_samples / n_estimators
        self.n_estimators = n_estimators
        self.features_rate = features_rate
        self.cou_samples = cou_samples
        self.loss = loss
        self.min_samples_split = min_samples_split
        self.shrinkage_value = learning_rate
        if (max_depth == None):
            self.max_depth = 10000
        else:
            self.max_depth = max_depth
    
    def get_predict_for_sample_and_one_tree(self, sample, tree_map):
        now_vertex = 0
        while (1):
            if (tree_map[str(now_vertex)][1][0] == 1):
                break
            if (sample[tree_map[str(now_vertex)][0][0]] >= tree_map[str(now_vertex)][0][1]):
                now_vertex = tree_map[str(now_vertex)][3][1]
            else:
                now_vertex = tree_map[str(now_vertex)][3][0]
        return tree_map[str(now_vertex)][2][0]
    
    def get_predict_for_sample(self, sample):
        answer = 0.0
        for i in range(len(self.trees_list)):
            answer += self.shrinkage_value * self.get_predict_for_sample_and_one_tree(sample, self.trees_list[i][1])
        return answer
    
    def update_grad_for_l2(self, x, tree_map):
        for i, sample in enumerate(x):
            self.now_grad[i] -= 2.0 * self.get_predict_for_sample_and_one_tree(sample, tree_map) * self.shrinkage_value
            
    def update_grad_for_l1(self, x, tree_map):
        for i, sample in enumerate(x):
            pred_for_sample = self.get_predict_for_sample_and_one_tree(sample, tree_map) * self.shrinkage_value
            self.y_balance[i] -= pred_for_sample
            if (self.y_balance[i] == 0.0):
                self.now_grad[i] = 0.00000001
            elif (self.y_balance[i] > 0.0):
                self.now_grad[i] = 1.0
            elif (self.y_balance[i] < 0.0):
                self.now_grad[i] = -1.0
        
        
    def predict_regression(self, x):
        y_pred = np.zeros(len(x))
        for i, sample in enumerate(x):
            y_pred[i] = self.get_predict_for_sample(sample)
        return y_pred
    
    def predict_regression_after_select(self, x):
        for i in reversed(xrange(x.shape[1])):
            if (i not in self.good_features):
                x = np.delete(x, i, axis = 1)
                
        y_pred = np.zeros(len(x))
        for i, sample in enumerate(x):
            y_pred[i] = self.get_predict_for_sample(sample)
        return y_pred
    
    def fit(self, x, y):
        self.pos_arr = np.arange(self.cou_samples)
        self.now_grad = np.array(y, dtype = np.float)
        self.trees_list = []
        self.y_balance = np.array(y, dtype = np.float)
        
        cart = CART(self.features_rate, self.min_samples_split, self.max_depth)
        
        self.feature_contribution = np.zeros((x.shape)[1])
        
        for i in range(self.n_estimators):
            np.random.shuffle(self.pos_arr)
            index = self.pos_arr[:self.cou_samples / 2]
            now_x = x[index]
            now_y = self.now_grad[index]
            
            cart.fit(now_x, now_y)
            if (i == 0 and self.loss == 'l2'):
                self.now_grad *= 2.0
            elif (i == 0 and self.loss == 'l1'):
                self.now_grad = np.zeros(len(self.now_grad))
                
            self.feature_contribution += cart.feature_contribution
            
            self.trees_list.append([self.shrinkage_value, cart.tree_map])
            
            if (self.loss == 'l2'):
                self.update_grad_for_l2(x, cart.tree_map)
            elif (self.loss == 'l1'):
                self.update_grad_for_l1(x, cart.tree_map)
                
        return self.trees_list
    
    def feature_selection_and_fit(self, x, y):
        self.fit(x, y)
        feature_probability = self.feature_contribution / np.sum(self.feature_contribution)
        self.good_features = []
        for i in reversed(xrange(x.shape[1])):
            if (feature_probability[i] < self.regularizator_value):
                x = np.delete(x, i, axis = 1)
            else:
                self.good_features.append(i)
                
        self.fit(x, y)
        
        

# Add noise features and remove them

In [70]:
feature1 = np.random.binomial(10000, 0.9, len(y_true))
feature2 = np.random.binomial(1000, 0.8, len(y_true))
feature3 = np.random.normal(0, 10, len(y_true))
feature4 = np.random.normal(21, 15, len(y_true))
feature5 = np.random.geometric(p=0.1, size=len(y_true))
feature6 = np.random.chisquare(15,len(y_true))
feature7 = np.random.uniform(-10,10,len(y_true))

In [71]:
data = np.insert(data, 0, feature7, axis = 1)
data = np.insert(data, 0, feature6, axis = 1)
data = np.insert(data, 0, feature5, axis = 1)
data = np.insert(data, 0, feature4, axis = 1)
data = np.insert(data, 0, feature3, axis = 1)
data = np.insert(data, 0, feature2, axis = 1)
data = np.insert(data, 0, feature1, axis = 1)

In [72]:
feature1 = np.random.binomial(10000, 0.9, len(test_y))
feature2 = np.random.binomial(1000, 0.8, len(test_y))
feature3 = np.random.normal(0, 10, len(test_y))
feature4 = np.random.normal(21, 15, len(test_y))
feature5 = np.random.geometric(p=0.1, size=len(test_y))
feature6 = np.random.chisquare(15,len(test_y))
feature7 = np.random.uniform(-10,10,len(test_y))

In [73]:
test_data = np.insert(test_data, 0, feature7, axis = 1)
test_data = np.insert(test_data, 0, feature6, axis = 1)
test_data = np.insert(test_data, 0, feature5, axis = 1)
test_data = np.insert(test_data, 0, feature4, axis = 1)
test_data = np.insert(test_data, 0, feature3, axis = 1)
test_data = np.insert(test_data, 0, feature2, axis = 1)
test_data = np.insert(test_data, 0, feature1, axis = 1)

In [74]:
params = {'n_estimators': 80, 'features_rate': 1.0, 'cou_samples': len(data), 'min_samples_split': 10,
                    'learning_rate' : 0.03, 'max_depth' : 4, 'loss' : 'l2', 'regularizator_value' : 0.007}
clf_without_select = modify_gb(**params)
clf_with_select = modify_gb(**params)

In [ ]:
clf_with_select.feature_selection_and_fit(x, y)
y_pred = clf_with_select.predict_regression_after_select(test_x)
print sk.metrics.mean_squared_error(y_pred, test_y)
print clf_with_select.feature_contribution
print clf_with_select.good_features

In [ ]:
print clf_with_select.feature_contribution / np.sum(clf_with_select.feature_contribution)

In [ ]:
clf_without_select.fit(x, y)
y_pred = clf_without_select.predict_regression(test_x)
print sk.metrics.mean_squared_error(y_pred, test_y)
print clf_without_select.feature_contribution

In [ ]:
y = test_y.copy()
y1 = test_y.copy()
estimators_list = np.linspace(1, 100, 100)
mse_select = []
mse_without_select = []
cou_samples = len(test_y)
y = np.array(y, dtype = np.float)
y1 = np.array(y1, dtype = np.float)
for i in range(0, 100):
    for j, sample in enumerate(test_data):
        y[j] -= clf_with_select.get_predict_for_sample_and_one_tree(sample, clf_with_select.trees_list[i][1]) * clf_with_select.shrinkage_value
        y1[j] -= clf_without_select.get_predict_for_sample_and_one_tree(sample, clf_without_select.trees_list[i][1]) * clf_without_select.shrinkage_value
    if (i + 1 in estimators_list):
        mse_select.append(np.sum(y ** 2.0) * 1.0 / cou_samples)
        mse_without_select.append(np.sum(y1 ** 2.0) * 1.0 / cou_samples)
    
    

        
estimators_list = np.asarray(estimators_list, dtype = np.float)
plt.figure(figsize = (8, 6), dpi = 80)
plt.xlabel('cou_estimators')
plt.ylabel('sklearn regression boosting error')
select, = plt.plot(estimators_list, mse_select, c = 'g', label = "select line")
without_select, = plt.plot(estimators_list, mse_without_select, c = 'y', label = "without_select")
plt.legend(handles = [select, without_select])
plt.show()